# ML Pipelines

## Data collection

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Datasets/Titanic-Dataset.csv')

In [3]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
447,448,1,1,"Seward, Mr. Frederic Kimber",male,34.0,0,0,113794,26.5500,NaN,S
431,432,1,3,"Thorneycroft, Mrs. Percival (Florence Kate White)",female,NaN,1,0,376564,16.1000,NaN,S
832,833,0,3,"Saad, Mr. Amin",male,NaN,0,0,2671,7.2292,NaN,C
527,528,0,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S
796,797,1,1,"Leader, Dr. Alice (Farnham)",female,49.0,0,0,17465,25.9292,D17,S


In [4]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace = True)

In [5]:
df.shape

(891, 8)

In [6]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

## Things we've to do:
- Handle missing values in 'age' and 'embarked'
- ohe for sex and embarked
- scaleing using MinMax
- Feature Selection
- Decision tree algo to train the model and predict value for new input

## Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size = 0.2,random_state = 0)

In [10]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S


In [11]:
X_train['Embarked'].nunique()

3

In [12]:
y_train.head()

140    0
439    0
817    0
378    0
491    0
Name: Survived, dtype: int64

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [14]:
# # Handling missing values
# trf1 = ColumnTransformer([('impute_age',SimpleImputer(),[2]),
#                         ('impute_embarked',SimpleImputer(strategy = 'most_frequent'),[6])], #most_frequent bcoz here values are letters 'S','C','Q'
#                          remainder = 'passthrough')
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [15]:
# #onehotencoding -> handle_unkown = ignore will ignore if any new category is found
# trf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse_output = False,dtype = np.int32,handle_unknown = 'ignore')),[1,6]],
#                          remainder = 'passthrough')

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')


In [16]:
# # Scaling
# from sklearn.preprocessing import MinMaxScaler
# #here columns are given from 0-8 because in the pipeline ohe is done first which increases columns from 7 -> 10(we are not dropping any column)
# trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))],
#                         remainder = 'passthrough')
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

NameError: name 'MinMaxScaler' is not defined

In [ ]:
# #feature Selection
# from sklearn.feature_selection import SelectKBest,chi2
# trf4 = SelectKBest(score_func = chi2, k = 8)
trf4 = SelectKBest(score_func=chi2,k=8)

In [ ]:
# #model Training
# from sklearn.tree import DecisionTreeClassifier
# trf5 = DecisionTreeClassifier()
trf5 = DecisionTreeClassifier()

## Create Pipeline

In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn import set_config
set_config(display='diagram')


In [ ]:
#first way to create a pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [ ]:
#alt way to create a pipeline
pipe2 = make_pipeline(trf2,trf2,trf3,trf4,trf5)

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8, score_func=<function chi2 at 0x1666ccae0>)),
                ('trf5', DecisionTreeClassifier())])

In [ ]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x1666ccae0>),
 'trf5': DecisionTreeClassifier()}

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
y_pred

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.6759776536312849

## Cross Validation using pipeline

In [ ]:
#calculates accuracy 'cv' times
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train,y_train, cv = 5, scoring = 'accuracy')

array([0.57342657, 0.62237762, 0.64084507, 0.61971831, 0.61971831])

In [ ]:
cross_val_score(pipe, X_train,y_train, cv = 5, scoring = 'accuracy').mean()

0.6152171771890081

## Exporting the pipeline

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

**NOTE**:
Go to predict_using_pipleline_titanic_ds.ipynb